In [33]:
import requests
import json
import pandas as pd
import plotly as plt
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import datetime

In [28]:
# DCurl stores DukasCopy URL
DCurl = "https://www.dukascopy.com/swiss/english/home/?utm_source=freeserv"

In [41]:
def getTime(t):
    return time.strftime('%Y-%d-%m %H:%M:%S', time.gmtime(t))

print(getTime(1578448320))

2020-08-01 01:52:00


In [31]:
def getDukasTime(soup):
    tdata = soup.find_all("span", {"id": "timeUpdate"})
    dtxt = tdata[0].text
    
    date_time_obj = datetime.datetime.strptime(dtxt, ' %a, %d %b %Y %H:%M:%S GMT')
    
    return date_time_obj

In [4]:
MetalsApiResponse = requests.get("http://metals-api.com/api/latest?access_key=anbne4hpic0ev3c2ymdyscbk5emkoni00qurxhpyrcvbuh2bu1j1yugbqbl17avu&base=USD&symbols = XAU") 

In [42]:
# Function to fetch Gold Data from MetalsAPI

def getMetalsApiData(MetalsApiResponse):
    print("MetalsAPI Connection " + str(MetalsApiResponse.status_code))
    
    # Get the result json
    result = MetalsApiResponse.json()
    
    # Store results in dataframe
    df = pd.DataFrame(result)
    df["timestamp"] = df["timestamp"].map(getTime)
    
    # Store only gold data and make time stamp the index
    df = df.loc[df.index == "XAU"]
    df = df.set_index("timestamp")
    
    return df
    
getMetalsApiData(MetalsApiResponse)
    



MetalsAPI Connection 200


,success,date,base,rates,unit
timestamp,,,,,
2020-08-01 09:10:00,True,2020-01-08,USD,1584.362611,per ounce


In [43]:
def getDukasCopyData():
    browser = webdriver.Chrome()
    browser.get(DCurl)
    DChtml = browser.page_source
    soup = BeautifulSoup(DChtml, "lxml")
    
    # Finding the exact table with all the required data
    data = soup.find_all("table", {"id": "list"})
    time = getDukasTime(soup)
    
    # Creating and Modifying dataframe with the table 
    DCDataFrame = pd.read_html(str(data))[0]
    
    DCDataFrame["timestamp"] = getDukasTime(soup)
    DCDataFrame = DCDataFrame.set_index("timestamp")
    DCDataFrame = DCDataFrame[DCDataFrame["Live"] == "XAU/USD"]
    
    browser.quit()
    return DCDataFrame
    
getDukasCopyData()

,Live,Bid,Ask,Spread
timestamp,,,,
2020-01-08 10:39:17,XAU/USD,1581.27,1581.5,23.0
